In [1]:
import librosa
import IPython.display as ipd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
import csv

In [2]:
#We’ll process dataset as per our requirements. 
#We’ll create a CSV file with the data we required.
header = 'Filename Chromagram RootMeanSquare LowEnergyBrightness SpectralCentroid Flatness Bandwidth InHarmonicity Rolloff ZeroCrossingRate'
for i in range(1,21):
        header += f' mfcc{i}'
header += ' label'
header = header.split()

In [ ]:
#If you have read the blog of features extraction 
#we’ll get 20 mfcc for given sampling rate because
#it is calculated for each frame so mfcc has 20 columns.
#Now, we’ll calculate all the features.
file = open('extractedmusicfeatureset.csv', 'w', newline='')
with file:
    writer = csv.writer(file)
    writer.writerow(header)
genres = 'blues classical country disco hiphop jazz metal pop reggae rock'.split()
for g in genres:
    for filename in os.listdir(f'./genres/{g}'):
        songname = f'./genres/{g}/{filename}'
        y, sr = librosa.load(songname, mono=True, duration=30)
        y = librosa.effects.harmonic(y)
        chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)#chromagram
        rms = librosa.feature.rms(y=y)#root-mean square
        spec_cont = librosa.feature.spectral_contrast(y=y, sr=sr)#low energy/brightness
        spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)#spectral centroid
        spec_flat = librosa.feature.spectral_flatness(y=y)#flatness
        spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)#bandwidth
        tone = librosa.feature.tonnetz(y=y, sr=sr)#In-Harmonicity
        rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)#rolloff
        zcr = librosa.feature.zero_crossing_rate(y)#zero-crossing rate
        mfcc = librosa.feature.mfcc(y=y, sr=sr)
        to_append = f'{filename} {np.mean(chroma_stft)} {np.mean(rms)} {np.mean(spec_cont)} {np.mean(spec_cent)} {np.mean(spec_flat)} {np.mean(spec_bw)} {np.mean(tone)} {np.mean(rolloff)} {np.mean(zcr)}'
        for e in mfcc:
            to_append += f' {np.mean(e)}'
        to_append += f' {g}'
        file = open('extractedmusicfeatureset.csv', 'a', newline='')
        with file:
            writer = csv.writer(file)
            writer.writerow(to_append.split())

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\scipy\sparse\lil.py:514: FutureWarning: future versions will not create a writeable array from broadcast_array. Set the writable flag explicitly to avoid this warning.
  if not j.flags.writeable or j.dtype not in (np.int32, np.int64):
C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\scipy\sparse\lil.py:514: FutureWarning: future versions will not create a writeable array from broadcast_array. Set the writable flag explicitly to avoid this warning.
  if not j.flags.writeable or j.dtype not in (np.int32, np.int64):


In [ ]:
##Split audio files goes here when refactored##

In [ ]:
# Dropping unneccesary columns
data = pd.read_csv('data_test_two.csv')
data = data.drop(['filename'],axis=1) #Filename is not required
data.head()

In [ ]:
features = ['Chromagram','RootMeanSquare','LowEnergyBrightness','SpectralCentroid','Flatness','Bandwidth','InHarmonicity','Rolloff','ZeroCrossingRate','mfcc1','mfcc2','mfcc3','mfcc4','mfcc5','mfcc6','mfcc7','mfcc8','mfcc9','mfcc10','mfcc11','mfcc12','mfcc13','mfcc14','mfcc15','mfcc16','mfcc17','mfcc18','mfcc19','mfcc20']

In [ ]:
X_prenorm = data[features]
pd.DataFrame(X_prenorm,columns=features).hist(figsize=(8,8), normed=True)

In [ ]:
#maybe should be X_prenorm.groupby instead
genre_count = data.groupby('label').agg({'label':'count'})['label']
print(genre_count)

In [ ]:
norm = Normalizer()
data[features] = norm.fit_transform(data[features])

In [ ]:
genre_groupby = data.groupby('label').agg('mean')
clusters = genre_groupby.reset_index().rename({'label':'cluster'}, axis=1)
clusters['method'] = 'Mean'

oldclusters = clusters
oldclusters

In [ ]:
data[features].hist(figsize=(8,8), normed=True)

In [ ]:
X = data[['Chromagram','RootMeanSquare','LowEnergyBrightness','SpectralCentroid','Flatness','Bandwidth','InHarmonicity','Rolloff','ZeroCrossingRate']]

Ks = np.arange(1, 20)
km = [KMeans(n_clusters=i, random_state=1986) for i in Ks]
score = [-km[i].fit(X).score(X) for i in range(len(km))]
Ks.dtype = int
plt.figure(figsize=(8,8))
plt.plot(Ks,score)
plt.xlim(0,18)
plt.grid(True)
plt.xlabel('K')
plt.ylabel('Error')
plt.title('Elbow Method')

In [ ]:
Ks = np.arange(2, 30)
kms = [KMeans(n_clusters=i, random_state=1986).fit(X) for i in Ks]
s_scores = [silhouette_score(X,kms[i].labels_) for i in range(2,len(Ks))]
plt.figure(figsize=(8,8))
plt.plot(Ks[2:len(Ks)],s_scores)
#plt.xlim(0,18)
plt.grid(True)
plt.xlabel('K')

In [ ]:
X = data[['Chromagram','RootMeanSquare','LowEnergyBrightness','SpectralCentroid','Flatness','Bandwidth','InHarmonicity','Rolloff','ZeroCrossingRate']]
km = KMeans(n_clusters=len(genre_groupby), n_init=10, random_state=1986)
km.fit(X)
labels = km.labels_
cluster_centers = km.cluster_centers_
labels_unique = np.unique(labels)
n_clusters_ = len(labels_unique)
km_clusters = pd.DataFrame(cluster_centers, columns=features)
km_clusters['cluster'] = ['KM{}'.format(l) for l in labels_unique]
km_clusters['method'] = 'KMeans'
oldclusters = pd.concat([oldclusters, km_clusters])
cols = ['cluster'] + features + ['method']
oldclusters = oldclusters[cols]


In [ ]:
newclusters = oldclusters

In [ ]:
newclusters

In [ ]:
plt.figure(figsize=(20,20))
sns.set(font_scale=1.5)
sns.heatmap(
    data=newclusters.loc[newclusters.method=='KMeans', features],
    cmap='Purples',
    annot=True
)
plt.ylabel("KMeansLabel")
plt.savefig('heatmap_all.png')

In [ ]:
#distances = [abs(np.linalg.norm(genre_coordinates-c)) for c in genre_coordinates_other]
### Assign K-Means Label to Descriptive Labels
descriptive_labels = ["Slow & Somber Acoustics", "Sad Instrumentals", "Upbeat Songs With Cheerful Vocals", 
                      "Fast & Danceable Instrumentals", "Fast, Upbeat & Cheerful Songs", "Happy & Upbeat Instrumentals",
                      "Aggressive, Fast Paced Acoustics", "Slow Happy Dance", "Happy & Slow", "Fast, Upbeat & Cheerful Acoustics"]

descriptive_labels = ["Slow & Somber Acoustics","Sad Instrumentals", "Upbeat Songs With Cheerful Vocals", 
                      "Fast & Danceable Instrumentals", "Fast, Upbeat & Cheerful Songs", "Happy & Upbeat Instrumentals",
                      "Aggressive, Fast Paced Acoustics", "Slow Happy Dance", "Happy & Slow", "Fast, Upbeat & Cheerful Acoustics"]
unique_labels = np.unique(labels)
translated_labels = dict(zip(unique_labels, descriptive_labels))
data['KMeansLabel'] = list(map(lambda x:translated_labels[x], labels))
################################################################################
#####################TRy other clustering algorithms############################
################################################################################

genre_count = data.groupby('label').agg({'label':'count'})['label']

# How many instances of each k-means cluster are there?
print("### Instances of KMeans Cluster ###")
print(data[['label', 'KMeansLabel']].groupby('KMeansLabel').agg('count'))

# Which cluster corresponds to the most instances per genre?
print("### Which cluster corresponds to the most instances per genre? ###")
print(data[['label', 'KMeansLabel']].groupby('label').agg(lambda x:x.value_counts().index[0]))

# Which genre corresponds to the most instances per cluster?
print("### Which genre corresponds to the most instances per cluster? ###")
print(data[['label', 'KMeansLabel']].groupby('KMeansLabel').agg(lambda x:x.value_counts().index[0]))

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
X_new = pca.fit_transform(X)
x,y = zip(*X_new)
data['x'] = x
data['y'] = y
components = pca.components_
explained_variance = pca.explained_variance_
explained_variance_ratio = pca.explained_variance_ratio_
singular_values = pca.singular_values_

In [ ]:
audio_components = dict(list(zip(features, zip(*np.round(components,2)))))
audio_components

In [ ]:
singular_values

In [ ]:
sns.set(font_scale=1.5)
sns.lmplot(data=data, x='x', y='y', hue='label', fit_reg=False, legend=True, size=8, palette='Set1',
           scatter_kws={'alpha':0.35, 's':25})
plt.savefig('pca_scatter_genres.png')
plt.title("Clustering by Genre, Reduced Dimension")

In [ ]:
sns.set(font_scale=1.5)
sns.lmplot(data=data, x='x', y='y', hue='KMeansLabel', fit_reg=False, legend=True, size=8, palette='Set1',
           scatter_kws={'alpha':0.35, 's':25})
plt.title("Clustering by KMeans Label, Reduced Dimension")
plt.savefig('pca_scatter_KM.png')

In [ ]:
##Confusion Matrix goes here when complete##